In [88]:
from glob import glob
from configs import *

In [89]:
# tag='joint|weather|ours'
# segment='-2|-3|0'
tag='ours|joint|weather'
segment='0|-2|-3'

In [90]:
tags = tag.split('|')
segs = segment.split('|')

In [97]:
def write_table(tag, segment, header=True, file=None):
    outputs = glob('outputs/%s_seg_%d*.txt' % (tag, segment))
    assert len(outputs) > 0
    outputs = sorted(outputs, key=lambda nm: int(nm.split('targ_')[1].replace('.txt', '')))
#     print(len(outputs))
#     print(outputs)
    results = []
    for sii, fname in enumerate(outputs):
        with open(fname) as fl:
            raw = fl.read()
            if 'EPOCH' in raw:
                parts = raw.split('EPOCH:')
                minind = 0
                minloss = 1000000000
                for eii, ent in enumerate(parts[1:]):
                    ent = ent.split('\n')[1:-1]
#                     print(ent)
                    msetest = float(ent[1].split(':')[1])
                    if msetest < minloss:
                        minloss = msetest
                        minind = eii
                ent = parts[minind]
                parts = ent.split('\n')[1:-1]
            else:
                parts = raw.split('\n')[:-1]

            kvs = [ent.split(':') for ent in parts]
            record = {}
            for key, val in kvs:
                record[key] = float(val)
            results.append(record)

    assert len(results) == len(outputs)
    
    def out(fl):
        segind = 0 if segment < 0 else segment
        if header:
            fl.write('\t'.join(['Metric'] + ['"%s"' % name[0] for name in SEGMENTS[segind]['locations']]))
            fl.write('\n')

        for field in ['MSE_TEST', 'MAPE', 'MSE_AVG']:
            values = []
            for entry in results:
                values.append('%.3f' % entry[field])
            fl.write('\t'.join([field + ' (%s)' % tag] + values))
            fl.write('\n')
    if file is None:
        with open('table_%s_%d.csv' % (tag, segment), 'w') as fl:
            out(fl)
    else: out(file)
#     print(results[0])

In [98]:
for tag, seg in zip(tags, segs):
    write_table(tag, int(seg))

In [99]:
with open('table_all.csv', 'w') as fl:
    for tii, (tag, seg) in enumerate(zip(tags, segs)):
        write_table(tag, int(seg), header=(tii == 0), file=fl)

In [100]:
nfields = 3
with open('table_all.csv') as fl:
    lines = fl.read().split('\n')[:-1]
    header = lines[0]
    data = lines[1:]
    assert len(data) % len(tags) == 0
    grouped = []
    for nii in range(nfields):
        for lii, line in enumerate(data):
            if lii % nfields == nii:
                grouped.append(line)
#     print(grouped)
    assert len(grouped) == len(data)
    
    with open('table_styled.csv', 'w') as fl:
        fl.write(header + '\n')
        fl.write('\n'.join(grouped))
    
#     for tii, (tag, seg) in enumerate(zip(tags, segs)):
#         write_table(tag, int(seg), header=(tii == 0), file=fl)